In [3]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras_tuner import Hyperband
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load and preprocess data
# Update the path to point to the correct location of processed_data.csv
data_path = "../0_DataPreparation/processed_data.csv"  # Ensure this path is correct
try:
    print(f"Loading data from {data_path}...")
    data = pd.read_csv(data_path)
    print("Data loaded successfully.")
except FileNotFoundError:
    raise FileNotFoundError(f"The file at {data_path} could not be found. Please check the file path.")

# Fill missing values and filter out rows with Umsatz = 0
print("Preprocessing data...")
data['Bewoelkung'] = data['Bewoelkung'].fillna(data['Bewoelkung'].mean())
data = data[data['Umsatz'] != 0]

# Define feature columns
print("Defining feature columns...")
feature_columns = [
    'KiWo', 'Is_Weekend', 'Temperature_Category',
    'Windgeschwindigkeit_Beaufort', 'Rain_Status',
    'Bewoelkung'
] + [col for col in data.columns if col.startswith('Warengruppe_')]
print(f"Feature columns: {feature_columns}")

# Split data into features and target
print("Splitting data into features and target...")
X = data[feature_columns].to_numpy()
y = data['Umsatz'].to_numpy()

# Split data into training and validation sets
print("Splitting data into training and validation sets...")
X_train_full, X_val_full, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
print("Scaling data...")
scaler = StandardScaler()
X_train_full = scaler.fit_transform(X_train_full)
X_val_full = scaler.transform(X_val_full)
print("Data scaling completed.")

# Define a custom model builder
def build_model(hp):
    print("Building model...")
    model = Sequential()
    input_shape = X_train_full.shape[1]  # Dynamically get the number of features

    # First hidden layer
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32), 
                    activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

    # Second hidden layer
    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))

    # Third hidden layer
    model.add(Dense(units=hp.Int('units_3', min_value=16, max_value=64, step=16), activation='relu'))

    # Output layer
    model.add(Dense(1, activation='linear'))

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='mse',
        metrics=['mae']
    )
    print("Model built successfully.")
    return model

# Set up the tuner
print("Setting up the tuner...")
tuner = Hyperband(
    build_model,
    objective='val_mae',
    max_epochs=20,
    factor=3,
    directory='my_dir',
    project_name='feature_tuning'
)
print("Tuner setup completed.")

# Early stopping
print("Defining early stopping...")
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Run the hyperparameter search
print("Starting hyperparameter search...")
tuner.search(
    X_train_full, y_train,
    validation_data=(X_val_full, y_val),
    callbacks=[early_stopping],
    verbose=1
)
print("Hyperparameter search completed.")

# Retrieve the best hyperparameters
print("Retrieving best hyperparameters...")
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps.values}")

# Build the model with the best hyperparameters
print("Building the model with the best hyperparameters...")
model = tuner.hypermodel.build(best_hps)

# Train the final model
print("Training the final model...")
history = model.fit(
    X_train_full, y_train,
    validation_data=(X_val_full, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)
print("Final model training completed.")

# Evaluate the model
print("Evaluating the model...")
val_loss, val_mae = model.evaluate(X_val_full, y_val, verbose=0)
print(f"Validation Loss: {val_loss}")
print(f"Validation MAE: {val_mae}")

# Compute Mean Absolute Percentage Error (MAPE)
print("Computing MAPE...")
y_val_pred = model.predict(X_val_full, verbose=0)
mape = 100 * np.mean(np.abs((y_val - y_val_pred.flatten()) / y_val))
print(f"Validation MAPE: {mape:.2f}%")


Loading data from ../0_DataPreparation/processed_data.csv...
Data loaded successfully.
Preprocessing data...
Defining feature columns...
Feature columns: ['KiWo', 'Is_Weekend', 'Temperature_Category', 'Windgeschwindigkeit_Beaufort', 'Rain_Status', 'Bewoelkung', 'Warengruppe_1', 'Warengruppe_2', 'Warengruppe_3', 'Warengruppe_4', 'Warengruppe_5', 'Warengruppe_6']
Splitting data into features and target...
Splitting data into training and validation sets...
Scaling data...
Data scaling completed.
Setting up the tuner...
Reloading Tuner from my_dir/feature_tuning/tuner0.json
Tuner setup completed.
Defining early stopping...
Starting hyperparameter search...
Hyperparameter search completed.
Retrieving best hyperparameters...
Best hyperparameters: {'units_1': 128, 'dropout_1': 0.2, 'units_2': 64, 'dropout_2': 0.4, 'units_3': 32, 'learning_rate': 0.01, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}
Building the model with the best

/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


233/233 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 17702.4492 - mae: 86.4008 - val_loss: 5390.6748 - val_mae: 43.0218
Epoch 2/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6195.9014 - mae: 51.1087 - val_loss: 6144.0825 - val_mae: 44.9810
Epoch 3/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6506.6729 - mae: 51.2786 - val_loss: 5286.6963 - val_mae: 43.0374
Epoch 4/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 7248.0400 - mae: 51.5577 - val_loss: 6059.5859 - val_mae: 48.5095
Epoch 5/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6551.3945 - mae: 51.1162 - val_loss: 5174.7651 - val_mae: 43.8441
Epoch 6/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6039.1958 - mae: 50.7330 - val_loss: 5504.8540 - val_mae: 42.7588
Epoch 7/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6170.1440 - mae: 50.0970 - val_loss: 4986.2217 - val_mae: 40.9840
Epoch 8/50
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6194.4785 - mae: 50.7273 - val_loss: 6802.2393 - val_mae: 55.3756
Ep

In [4]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import StandardScaler

# Paths to the files
processed_data_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/processed_data.csv"
sample_submission_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/sample_submission.csv"
final_submission_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/final_submission.csv"
model_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/3_Model/nn_model.h5"

# Load the processed data and sample submission
processed_data = pd.read_csv(processed_data_path)
sample_submission = pd.read_csv(sample_submission_path)

# Dynamically rebuild feature columns
feature_columns = [
    'KiWo', 'Is_Weekend', 'Temperature_Category',
    'Windgeschwindigkeit_Beaufort', 'Rain_Status',
    'Bewoelkung', 'Warengruppe_1', 'Warengruppe_2',
    'Warengruppe_3', 'Warengruppe_4', 'Warengruppe_5', 'Warengruppe_6'
]

# Extract features for prediction
X_new = processed_data[feature_columns].apply(pd.to_numeric, errors='coerce').fillna(0).to_numpy(dtype=np.float64)

# Load the trained neural network model
model = load_model(
    model_path,
    custom_objects={'mse': MeanSquaredError()}  # Ensure compatibility with saved model
)

# Normalize features using the same scaler used during training
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)  # Use saved scaler if available

# Predict the output using the trained neural network
y_pred = model.predict(X_new).flatten()

# Add predictions to the processed data DataFrame
processed_data['Predicted_Umsatz'] = y_pred

# Merge predictions with sample submission to ensure matching structure
final_submission = sample_submission[['id']].copy()
final_submission = final_submission.merge(
    processed_data[['ID', 'Predicted_Umsatz']],
    how='left',
    left_on='id',
    right_on='ID'
)

# Drop the redundant 'ID' column and rename 'Predicted_Umsatz' to 'Umsatz'
final_submission.drop(columns=['ID'], inplace=True)
final_submission.rename(columns={'Predicted_Umsatz': 'Umsatz'}, inplace=True)

# Replace null values in the Umsatz column with 0
final_submission['Umsatz'] = final_submission['Umsatz'].fillna(0)

# Save the final submission file
os.makedirs(os.path.dirname(final_submission_path), exist_ok=True)
final_submission.to_csv(final_submission_path, index=False)

print(f"Final submission saved to: {final_submission_path}")


129/439 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step

439/439 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step
Final submission saved to: /workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/final_submission.csv
